In [2]:
# import packages
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import Bio
import statsmodels.api as sm
import re

Import all excelsheets as a dataframe and store it in a dictionary and rename keys to exclude 'TPP_' from beginning

In [3]:
data_dir = "./Bioinfo_data_extracted"
file_list = os.listdir(data_dir)
raw_data ={os.path.splitext(filename)[0]: pd.read_excel(os.path.join(data_dir, filename), header=0) 
        for filename in file_list}

raw_data = {key[4:]: value for key, value in raw_data.items()} # remove 'TPP_' from the keys

ValueError: Excel file format cannot be determined, you must specify an engine manually.

Rename columns of each dataframe to have it consistent.
Drop all not important columns

In [1]:
#list copied from col_list and manually adjusted
drop_list = ['norm_FC_TMT126_H.sapiens_HEK293_PP018224', 'norm_FC_TMT127L_H.sapiens_HEK293_PP018224', 'norm_FC_TMT127H_H.sapiens_HEK293_PP018224', 
            'norm_FC_TMT128L_H.sapiens_HEK293_PP018224', 'norm_FC_TMT128H_H.sapiens_HEK293_PP018224', 'norm_FC_TMT129L_H.sapiens_HEK293_PP018224', 
            'norm_FC_TMT129H_H.sapiens_HEK293_PP018224', 'norm_FC_TMT130L_H.sapiens_HEK293_PP018224', 'norm_FC_TMT130H_H.sapiens_HEK293_PP018224', 
            'norm_FC_TMT131L_H.sapiens_HEK293_PP018224', 'a_H.sapiens_HEK293_PP018224', 'b_H.sapiens_HEK293_PP018224', 
            'inflPoint_H.sapiens_HEK293_PP018224', 'slope_H.sapiens_HEK293_PP018224', 'plateau_H.sapiens_HEK293_PP018224', 
            'R_sq_H.sapiens_HEK293_PP018224', 'plot', 'protein_identified_in_H.sapiens_HEK293_PP018224', 'model_converged_H.sapiens_HEK293_PP018224', 
            'sufficient_data_for_fit_H.sapiens_HEK293_PP018224','numSpec_H.sapiens_HEK293_PP018224','plot_link']

col_name = pd.read_csv('C:/Users/marik/Downloads/temp column name.csv')
col_list = col_name.columns.tolist()
key_list = list(raw_data.keys())

# Rename columns based on col_list (equal to drop_list, except for elements not included)
for n in range(len(key_list)):
    old_cols = list(raw_data[key_list[n]].columns)
    col_list = col_name[:len(old_cols)]  # Ensure col_name has enough elements
    rename_dict = dict(zip(old_cols, col_list))
    raw_data[key_list[n]] = raw_data[key_list[n]].rename(columns=rename_dict)

# Drop columns in drop_list
for n in range(len(key_list)):
    raw_data[key_list[n]] = raw_data[key_list[n]].drop(columns = drop_list, axis=1)

# Rename columns to be consistent across all dataframes
raw_data[key_list[0]].head()
for n in range(len(key_list)):
    raw_data[key_list[n]] = raw_data[key_list[n]].rename(columns={
        'meltPoint_H.sapiens_HEK293_PP018224': 'meltPoint', 'Proteinname_H.sapiens_HEK293_PP018224': 'Proteinname'})

NameError: name 'pd' is not defined

Create a new column for each dataframe in the dictionary and add the name of the sample (key name) to it.

In [ ]:
re.replace('What to replace', 'Replace with', string of text to search)

In [198]:
data = {}
for i in range(len(raw_data.keys())):
    df = raw_data[list(raw_data.keys())[i]].copy()
    if df.columns[0] != 'Sample':
        df.insert(loc=0, column = 'Sample', 
                  value = pd.Series(str(list(raw_data.keys())[i]), index = df.index))
        data[list(raw_data.keys())[i]] = df
    else:
        data[list(raw_data.keys())[i]] = df



Create one big joined dataframe from the dictionary

In [199]:

data_human = pd.concat(data.values(), axis = 0, ignore_index=True, join='inner')
data_human.head()

,Sample,Protein_ID,meltPoint,Proteinname,gene_name,uniprot_ac,auc
0,H.sapiens_A549_P013697,A0AVK6_E2F8,NaN,Transcription factor E2F8,E2F8,A0AVK6,0.673865
1,H.sapiens_A549_P013697,A0AVT1_UBA6,49.407548,Ubiquitin-like modifier-activating enzyme 6,UBA6,A0AVT1,0.378148
2,H.sapiens_A549_P013697,A0JNW5_UHRF1BP1L,57.121525,UHRF1-binding protein 1-like,UHRF1BP1L,A0JNW5,0.641599
3,H.sapiens_A549_P013697,A0MZ66_KIAA1598,54.592113,Shootin-1,KIAA1598,A0MZ66,0.580735
4,H.sapiens_A549_P013697,A1A4S6_ARHGAP10,55.426149,Rho GTPase-activating protein 10,ARHGAP10,A1A4S6,0.613258


Identify healthy and desease cell lines

In [201]:
print(data_human['Sample'].unique())

'H.sapiens_A549_P013697' 'H.sapiens_A549_P013701'
 'H.sapiens_colon_cancer_spheroids_R1_P016043'
 'H.sapiens_colon_cancer_spheroids_R2_P016046' 'H.sapiens_HEK293_PP018221'
 'H.sapiens_HEK293_PP018224' 'H.sapiens_hepatocytes_P013190'
 'H.sapiens_hepatocytes_P013194' 'H.sapiens_hepatocytes_P013553'
 'H.sapiens_hepatocytes_P013558' 'H.sapiens_Jurkat_P015963'
 'H.sapiens_Jurkat_P015964' 'H.sapiens_K562_cells_R1_P018450'
 'H.sapiens_K562_cells_R2_P018894' 'H.sapiens_K562_lysate_R1_P019670'
 'H.sapiens_K562_lysate_R2_P019785' 'H.sapiens_K562_P013109'
 'H.sapiens_K562_P013110' 'H.sapiens_K562_P018889'
 'H.sapiens_K562_P018890' 'H.sapiens_lung_fibroblasts_P017092'
 'H.sapiens_lung_fibroblasts_P017095'
 'human_body_fluids_blood_plasma_P014819'
 'human_body_fluids_breast_milk_P014818' 'human_body_fluids_CSF_P014820'
 'human_body_fluids_saliva_P014425'
 'human_body_fluids_seminal_plasma_P014428'

['H.sapiens_A549_P013697' 'H.sapiens_A549_P013701'
 'H.sapiens_colon_cancer_spheroids_R1_P016043'
 'H.sapiens_colon_cancer_spheroids_R2_P016046' 'H.sapiens_HEK293_PP018221'
 'H.sapiens_HEK293_PP018224' 'H.sapiens_hepatocytes_P013190'
 'H.sapiens_hepatocytes_P013194' 'H.sapiens_hepatocytes_P013553'
 'H.sapiens_hepatocytes_P013558' 'H.sapiens_Jurkat_P015963'
 'H.sapiens_Jurkat_P015964' 'H.sapiens_K562_cells_R1_P018450'
 'H.sapiens_K562_cells_R2_P018894' 'H.sapiens_K562_lysate_R1_P019670'
 'H.sapiens_K562_lysate_R2_P019785' 'H.sapiens_K562_P013109'
 'H.sapiens_K562_P013110' 'H.sapiens_K562_P018889'
 'H.sapiens_K562_P018890' 'H.sapiens_lung_fibroblasts_P017092'
 'H.sapiens_lung_fibroblasts_P017095'
 'human_body_fluids_blood_plasma_P014819'
 'human_body_fluids_breast_milk_P014818' 'human_body_fluids_CSF_P014820'
 'human_body_fluids_saliva_P014425'
 'human_body_fluids_seminal_plasma_P014428']
